In [25]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score

### Load data

In [26]:
df = pd.read_csv("../data/clean/main_df.csv")

In [27]:
# Drop unnecessary/unfit columns

# Convert "playoff" column to binary (Y: 1, N: 0)
df["playoff"] = df["playoff"].map({"Y": 1, "N": 0})

def map_strings_to_int(df, col):
    """
    Maps string values to integers
    """
    values = df[col].unique()
    mapping = {value: i for i, value in enumerate(values)}
    df[col] = df[col].map(mapping)

map_strings_to_int(df, "tmID")
map_strings_to_int(df, "playerID")
map_strings_to_int(df, "coachID")
map_strings_to_int(df, "pos")

df.to_csv("../data/clean/pre.csv", index=False)


### Training and evaluation function

In [28]:
from dataclasses import dataclass

@dataclass
class Result:
    year: int
    accuracy: float
    auc: float

results = {}

def classification(model, teams_df, min_year, max_year):
    for i in range(min_year, max_year + 1):
        teams_df_train = teams_df[teams_df['year'] < i]
        teams_df_test = teams_df[teams_df['year'] == i]

        X_train = teams_df_train.drop("playoff", axis=1)  # Features
        y_train = teams_df_train["playoff"]  # Target variable

        X_test = teams_df_test.drop("playoff", axis=1)  # Features
        y_test = teams_df_test["playoff"]  # Target variable

        print(f"\nTrain/Test size for year={i}:", X_train.shape, X_test.shape, y_train.shape, y_test.shape)

        model.fit(X_train, y_train)

        # Predict the response for the test dataset
        y_pred = model.predict(X_test)

        print(f"\nyear = {i}, Classification Report:\n", classification_report(y_test, y_pred))
        print(f"\nyear = {i}, Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
        print(f"\nyear = {i}, AUC: ", roc_auc_score(y_test, y_pred))
        print(f"\nyear = {i}, Accuracy: ", accuracy_score(y_test, y_pred))

        result = Result(
            year=i,
            accuracy=accuracy_score(y_test, y_pred),
            auc=roc_auc_score(y_test, y_pred),
        )

        if (str(model) not in results):
            results[str(model)] = []
        results[str(model)].append(result)

min_year = 2
max_year = df['year'].max()

### Decision Tree

In [30]:
model = DecisionTreeClassifier(random_state=42)
classification(model, df, min_year, max_year)


Train/Test size for year=2: (222, 93) (235, 93) (222,) (235,)


ValueError: Input X contains NaN.
DecisionTreeClassifier does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

### SVC

In [29]:
model = SVC(random_state=42)
classification(model, df, min_year, max_year)


Train/Test size for year=2: (222, 93) (235, 93) (222,) (235,)


ValueError: Input X contains NaN.
SVC does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

### Logistic Regression

In [31]:
model = LogisticRegression(random_state=42, max_iter=100000, solver='newton-cg')
classification(model, df, min_year, max_year)


Train/Test size for year=2: (222, 93) (235, 93) (222,) (235,)


ValueError: Input X contains NaN.
LogisticRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

### Random Forest

In [32]:
model = RandomForestClassifier(random_state=42)
classification(model, df, min_year, max_year)


Train/Test size for year=2: (222, 93) (235, 93) (222,) (235,)


ValueError: Input X contains NaN.
RandomForestClassifier does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

### K Neighbors

In [33]:
model = KNeighborsClassifier()
classification(model, df, min_year, max_year)


Train/Test size for year=2: (222, 93) (235, 93) (222,) (235,)


ValueError: Input X contains NaN.
KNeighborsClassifier does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

# Results

In [ ]:
from pprint import pprint

# This is ugly
print("train/test results per model for the last year of data:")
pprint(dict(sorted(dict(map(lambda i: (i[0], i[1].accuracy), dict(map(lambda i: (i[0], i[1][-1]), results.items())).items())).items(), key=lambda i: i[1], reverse=True)))

for model in results:
    model_results = results[model]

    if any(map(lambda r: r.accuracy == 1, model_results)):
        print(f"\n{model} has 100% accuracy for at least one year, possible data leakage")